

```
# O objetivo é montar um arquivo .csv para análise de dados, o arquivo deve conter:
- Curso do estudante
- Estudante achou a disciplina útil?
- Semestre em que o feedback foi coletado
- Texto do feedback
- Sentimento do feedback quando existir
```



In [21]:
# Imports

import pandas as pd
import numpy as np

# Drive
from google.colab import drive

In [112]:
# Conectando ao google drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 avaliacao_da_disciplina_2022_1.xlsx   RePro.csv
 bitcoin_wallet_keywords.png	       repro-main.zip
'Colab Notebooks'		       review_repro_preprocessado.csv
 dataset_professora.csv		       reviews_ptbr.csv
 dataset_professora.gsheet	       tcc_analise_sentimento1.xlsx
 monero_wallet_keywords.png	       tw_pt.csv
 olist_preprocessado.csv


In [113]:
# Lendo a primeira planilha em um dataframe
tcc_analise_sentimento1 = pd.read_excel("/content/drive/MyDrive/tcc_analise_sentimento1.xlsx", engine="openpyxl", sheet_name=None)
# Apresentando abas carregadas
print("Abas carregadas:", tcc_analise_sentimento1.keys())

Abas carregadas: dict_keys(['analise_ensino_remoto', 'analise_primeiro_semestre', 'avaliacao_da_disciplina', 'licenciatura'])


In [114]:
# Lendo a segunda planilha em um dataframe
avaliacao_disciplina_2022_1 = pd.read_excel("/content/drive/MyDrive/avaliacao_da_disciplina_2022_1.xlsx", engine="openpyxl", sheet_name=None)
# Apresentando abas carregadas
print("Abas carregadas:", avaliacao_disciplina_2022_1.keys())

Abas carregadas: dict_keys(['avaliacao_disciplina'])




```
# Criando dataframes para cada tab da planilha
```



In [115]:
# Criando dataframes para cada tab da planilha tcc_analise_sentimento1
analise_ensino_remoto = tcc_analise_sentimento1['analise_ensino_remoto']
analise_primeiro_semestre = tcc_analise_sentimento1['analise_primeiro_semestre']
avaliacao_da_disciplina = tcc_analise_sentimento1['avaliacao_da_disciplina']
licenciatura = tcc_analise_sentimento1['licenciatura']

# Criando dataframes para cada tab da planilha avaliacao_da_disciplina_2022_1
avaliacao_disciplina = avaliacao_disciplina_2022_1['avaliacao_disciplina']



```
# Agora que o novo dataframe está criado vamos preencher com os dados
fornecidos pela professora
```



In [156]:
# Criando um DataFrame vazio para armazenar os feedbacks organizados
df_coleta_insight = pd.DataFrame(columns=['semestre', 'curso', 'achou_util', 'feedback', 'sentimento'])

# Função para limpar colunas desnecessárias
def concat_coluna(coluna_feedback):
    global df_coleta_insight
    global tcc_analise_sentimento1
    global avaliacao_disciplina_2022_1
    avaliacao_da_disciplina = tcc_analise_sentimento1['avaliacao_da_disciplina'].copy()
    licenciatura = tcc_analise_sentimento1['licenciatura'].copy()
    avaliacao_disciplina = avaliacao_disciplina_2022_1['avaliacao_disciplina'].copy()

    colunas_descartar = [
        'Descreva como foi esse primeiro semestre na Universidade de Brasilia  (escreva no mínimo 250 caracteres e no máximo 500)',
        'Descreva o que você está achando desde modelo de ensino remoto (escreva no mínimo 250 caractérers e no máximo 500)',
        'Descreva o que você achou desta disciplina, você acha que devemos oferecer para os calouros no próximo semestre (escreva no mínimo 250 caracteres e no máximo 500)',
        'Gostaria de deixar alguma sugestão para o próximo Programa de Acolhiment@ d@s Calour@s no semestre 2021.1? ',
        'Gostaria de deixar alguma sugestão para o próximo Programa de Acolhiment@ d@s Calour@s no semestre 2022.2? '
    ]


    # Definindo os semestres para cada DataFrame
    avaliacao_da_disciplina['semestre'] = '2021.1'
    licenciatura['semestre'] = '2021.1'
    avaliacao_disciplina['semestre'] = '2022.1'

    # Renomeando colunas para padronizar os DataFrames
    renomeacoes = {
        'Qual o seu curso': 'curso',
        'TIPO': 'sentimento',
        'Você achou útil essa disciplina': 'achou_util',
        coluna_feedback: 'feedback'
    }

    avaliacao_da_disciplina.rename(columns=renomeacoes, inplace=True)
    avaliacao_disciplina.rename(columns=renomeacoes, inplace=True)
    licenciatura.rename(columns=renomeacoes, inplace=True)

    dataframes = [avaliacao_da_disciplina, licenciatura, avaliacao_disciplina]
    for df in dataframes:
        for coluna in colunas_descartar:
            if coluna in df.columns:
                df.drop(columns=coluna, inplace=True)
        df_coleta_insight = pd.concat([df_coleta_insight, df], ignore_index=True)
    # Limpando
    limpeza = ['Data', 'alunos']
    df_coleta_insight.drop(columns=limpeza, inplace=True)




In [157]:
concat_coluna('Descreva como foi esse primeiro semestre na Universidade de Brasilia  (escreva no mínimo 250 caracteres e no máximo 500)')
concat_coluna('Descreva o que você está achando desde modelo de ensino remoto (escreva no mínimo 250 caractérers e no máximo 500)')
concat_coluna('Descreva o que você achou desta disciplina, você acha que devemos oferecer para os calouros no próximo semestre (escreva no mínimo 250 caracteres e no máximo 500)')
concat_coluna('Gostaria de deixar alguma sugestão para o próximo Programa de Acolhiment@ d@s Calour@s no semestre 2021.1? ')
concat_coluna('Gostaria de deixar alguma sugestão para o próximo Programa de Acolhiment@ d@s Calour@s no semestre 2022.2? ')

if df_coleta_insight['feedback'].isnull().any():
    df_coleta_insight.dropna(subset=['feedback'], inplace=True)

df_coleta_insight.reset_index(drop=True, inplace=True)
df_coleta_insight.shape

(368, 5)

In [158]:
# Salvando no drive um dataset contendo todos os registros disponibilizados pela professora com os campos abaixo

df_coleta_insight.to_csv('dataset_professora.csv', index= False, columns= ['semestre', 'curso', 'achou_util', 'feedback', 'sentimento'])